## READ FILE


In [8]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# Define file path and sheet names
file_path = "ULSP-instancesR.xlsx"
sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]

# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Insert dummy period 0
    demand_forecast = np.insert(df["Demand Forecast"].to_numpy(), 0, 0)
    setup_cost = np.insert(df["Setup Cost"].to_numpy(), 0, 0)
    production_cost = np.insert(df["Production cost"].to_numpy(), 0, 0)
    holding_cost = np.insert(df["Holding cost"].to_numpy(), 0, 0)
    backlogging_cost = np.insert(df["Backlogging cost"].to_numpy(), 0, 0)

    T = df.shape[0]
    print(f"Number of periods: {T}")

    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")

    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  
    b = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="b")  # Amount backlogged at the end of period t

    # Objective function
    model.setObjective(
            quicksum(setup_cost[t] * y[t] for t in range(1, T+1)) +
            quicksum(production_cost[t] * x[t] for t in range(1, T+1)) +
            quicksum(holding_cost[t] * S[t] for t in range(1, T+1)) +
            quicksum(backlogging_cost[t] * b[t] for t in range(1, T+1)),
            GRB.MINIMIZE
        )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")
    model.addConstr(b[0] == 0, name="no_backlogging0")
    model.addConstr(b[T] == 0, name="no_backloggingT")

    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] - b[t-1] == demand_forecast[t] + S[t] - b[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= (quicksum(demand_forecast[m] for m in range(t, T+1)) + b[t-1]) * y[t], name=f"setup_constraint_{t}")

    # Solve model
    model.optimize()

    # Check if the model found an optimal solution
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        # Create a temporary DataFrame for the current sheet results
        sheet_results = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])
        for t in range(1, T+1):
            # Store the results for the current sheet
            sheet_results = pd.concat([sheet_results, pd.DataFrame([{
                "Sheet": sheet_name,
                "Period": t,
                "y": y[t].X,
                "x": x[t].X,
                "S": S[t].X,
                "b": b[t].X
            }])], ignore_index=True)
        
        # Concatenate the results for all sheets
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)

    else:
        print(f"No optimal solution found for sheet {sheet_name}.")

# Save results to a CSV file
results_df.to_csv("optimization_results.csv", index=False)
print("\nResults have been saved to 'optimization_results.csv'.")


Processing sheet: 6-periods (1)
Number of periods: 6
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))



CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 28 columns and 34 nonzeros
Model fingerprint: 0x38a97cbc
Model has 6 quadratic constraints
Variable types: 21 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolved: 22 rows, 37 columns, 68 nonzeros
Presolved model has 10 SOS constraint(s)
Variable types: 26 continuous, 11 integer (11 binary)
Found heuristic solution: objective 249607.00090

Root relaxation: objective 2.755488e+04, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{
C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, sheet_results], ignore_index=True)
C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entri


CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 52 columns and 64 nonzeros
Model fingerprint: 0x2968c9fc
Model has 12 quadratic constraints
Variable types: 39 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 6e+03]
  Objective range  [2e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+03]
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolved: 46 rows, 79 columns, 146 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 56 continuous, 23 integer (23 binary)
Found heuristic solution: objective 185742.00222

Root relaxation: objective 4.101258e+04, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{
C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


...

C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


     0     2 315702.183    0   99 344845.000 315702.183  8.45%     -    0s
H  122   132                    344343.00000 319489.347  7.22%   1.8    0s
H  159   188                    336774.00000 319489.347  5.13%   1.7    0s
H  175   188                    336363.00000 319489.347  5.02%   1.7    0s
*  182   188              39    335189.00000 319489.347  4.68%   1.7    0s
*  193   184              41    333480.00000 319489.347  4.20%   1.7    0s
*  194   184              41    332221.00000 319489.347  3.83%   1.7    0s
*  372   245              35    332122.00000 322042.574  3.03%   1.9    0s
H  487   331                    331869.00000 322329.814  2.87%   2.1    0s
H  605   392                    331864.00000 323265.348  2.59%   2.2    0s
* 1115   638              37    331809.00000 324497.911  2.20%   2.3    0s
* 1311   698              37    331503.00000 324612.143  2.08%   2.3    0s
* 1316   686              35    331404.00000 324612.143  2.05%   2.3    0s
H 1603   719             

C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{



Root relaxation: objective 3.993129e+05, 467 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 399312.901    0  143 6820037.01 399312.901  94.1%     -    0s
     0     0 504353.299    0  125 6820037.01 504353.299  92.6%     -    0s
H    0     0                    1838121.0000 507206.134  72.4%     -    0s
H    0     0                    1574973.0000 507206.134  67.8%     -    0s
H    0     0                    1466521.0000 507206.134  65.4%     -    0s
     0     0 507206.134    0  126 1466521.00 507206.134  65.4%     -    0s
     0     0 521723.983    0  158 1466521.00 521723.983  64.4%     -    0s
     0     0 522458.742    0  158 1466521.00 522458.742  64.4%     -    0s
     0     2 524328.619    0  158 1466521.00 524328.619  64.2%     -    0s
H   33    40                    1413242.0000 536962.423  62.0%   2.6    0s
H   37   

C:\Users\ricca\AppData\Local\Temp\ipykernel_5848\331243633.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


In [2]:
import matplotlib.pyplot as plt

# Generate and save histograms for each sheet
for sheet_name in results_df["Sheet"].unique():
    sheet_data = results_df[results_df["Sheet"] == sheet_name]

    plt.figure(figsize=(8, 5))
    plt.bar(sheet_data["Period"], sheet_data["y"], color='blue', alpha=0.7)
    plt.xlabel("Period")
    plt.ylabel("Inventory Level (S)")
    plt.title(f"Inventory Levels for {sheet_name}")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Save histogram as an image
    hist_filename = f"histogram_{sheet_name.replace(' ', '_')}.png"
    plt.savefig(hist_filename)
    print(f"Histogram saved: {hist_filename}")
    plt.close()

Histogram saved: histogram_6-periods_(1).png
Histogram saved: histogram_6-periods_(2).png
Histogram saved: histogram_12-periods_(1).png
Histogram saved: histogram_12-periods_(2).png
Histogram saved: histogram_24-periods_(1).png
Histogram saved: histogram_24-periods_(2).png
Histogram saved: histogram_52-periods_(1).png
Histogram saved: histogram_52-periods_(2).png
Histogram saved: histogram_104-periods_(1).png
Histogram saved: histogram_104-periods_(2).png
